# 重新定义大模型和智能体

## 多轮对话

使用`messages`管理多轮对话的记忆。<br>
你也可以在`chat`函数中通过参数`k`限制希望保留的对话轮次，但这不影响`messages`记录完整的对话过程。

In [1]:
from textlong.llm import fake_llm, zhipu, qwen
from textlong.desk import Desk
from textlong.io import log, alog

d = Desk(zhipu)

In [2]:
d.get_chat_memory()

[]

### 默认记住10轮

In [3]:
log(d.chat("你能帮我写一首关于兔子做梦的四句儿歌?"))

的诗句，以下是一首关于兔子做梦的四句儿歌：


梦里花开香满地，
兔子跳跃入梦境。
阳光照耀心欢畅，
梦境醒来笑盈盈。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 63378887】


'的诗句，以下是一首关于兔子做梦的四句儿歌：\n\n\n梦里花开香满地，\n兔子跳跃入梦境。\n阳光照耀心欢畅，\n梦境醒来笑盈盈。'

In [3]:
await alog(d.achat("你能帮我写一首关于兔子做梦的四句儿歌?"))

当然可以！以下是一首关于兔子做梦的四句儿歌：

梦里花开兔子跳，
彩虹桥上唱歌谣。
兔子王国笑声闹，
梦醒花开满枝头。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 86412652】


'当然可以！以下是一首关于兔子做梦的四句儿歌：\n\n梦里花开兔子跳，\n彩虹桥上唱歌谣。\n兔子王国笑声闹，\n梦醒花开满枝头。'

In [6]:
d.get_chat_memory()

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '当然可以，以下是一首关于兔子做梦的四句儿歌：\n\n梦里花开兔子跳，\n彩虹桥上歌声飘。\n月亮弯弯照小屋，\n兔子笑醒梦如潮。'}]

In [7]:
log(d.chat("换成小羊吧"))

当然可以，这是关于小羊做梦的四句儿歌：

梦中绿草小羊悠，
蓝天白云梦无边。
小溪潺潺唱夜曲，
醒来小羊笑眯眯。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 41854805】


'当然可以，这是关于小羊做梦的四句儿歌：\n\n梦中绿草小羊悠，\n蓝天白云梦无边。\n小溪潺潺唱夜曲，\n醒来小羊笑眯眯。'

### 限制记忆中的对话轮数

In [8]:
log(d.chat("改为在蓝天上"))

蓝天上，小羊梦飞翔，以下是关于小羊在蓝天上做梦的四句儿歌：

蓝天下，羊儿梦飞扬，
白云间，小羊心欢畅。
风吹云动梦初醒，
阳光里，小羊笑洋洋。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 72807792】


'蓝天上，小羊梦飞翔，以下是关于小羊在蓝天上做梦的四句儿歌：\n\n蓝天下，羊儿梦飞扬，\n白云间，小羊心欢畅。\n风吹云动梦初醒，\n阳光里，小羊笑洋洋。'

调用时可以切换模型，例如用来模拟和查看提示语的 `fake_llm`：

In [12]:
d.messages

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '当然可以！以下是一首关于兔子做梦的四句儿歌：\n\n梦里花丛兔子跳，\n彩虹桥上笑哈哈。\n月亮姐姐讲故事，\n兔子宝宝入梦乡。'},
 {'role': 'user', 'content': '换成小羊吧'},
 {'role': 'assistant',
  'content': '当然可以，这是一首关于小羊做梦的四句儿歌：\n\n梦里草原小羊跳，\n白云朵朵伴歌谣。\n星星姐姐讲故事，\n小羊宝宝梦香甜。'},
 {'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant',
  'content': '好的，这是一首关于小羊在蓝天上做梦的四句儿歌：\n\n梦里蓝天小羊翔，\n白云做伴任飞翔。\n彩虹桥下牧歌起，\n小羊梦境最欢畅。'}]

In [9]:
d.get_chat_memory(1)

[{'role': 'user', 'content': '改为在蓝天上'},
 {'role': 'assistant',
  'content': '蓝天上，小羊梦飞翔，以下是关于小羊在蓝天上做梦的四句儿歌：\n\n蓝天下，羊儿梦飞扬，\n白云间，小羊心欢畅。\n风吹云动梦初醒，\n阳光里，小羊笑洋洋。'}]

In [10]:
log(d.chat("改为在水塘", llm=qwen, k=1))

水塘边，小羊梦悠扬，
碧波里，倒映着天光。
芦苇轻摇曲中睡，
涟漪轻泛，小羊梦更香。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 90184494】


'水塘边，小羊梦悠扬，\n碧波里，倒映着天光。\n芦苇轻摇曲中睡，\n涟漪轻泛，小羊梦更香。'

In [11]:
d.get_chat_memory(1)

[{'role': 'user', 'content': '改为在水塘'},
 {'role': 'assistant',
  'content': '水塘边，小羊梦悠扬，\n碧波里，倒映着天光。\n芦苇轻摇曲中睡，\n涟漪轻泛，小羊梦更香。'}]

### 多轮对话 + 回调工具

In [12]:
from textlong.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.desk import Desk
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

messages = []
d = Desk(qwen, toolkits=[get_current_weather], tools=[convert_to_openai_tool(get_current_weather)])

In [13]:
print(d.toolkits)

[StructuredTool(name='get_current_weather', description='获取城市的天气情况', args_schema=<class 'pydantic.main.get_current_weatherSchema'>, func=<function get_current_weather at 0x1235223b0>), StructuredTool(name='python_code', description='回答关于[]等数据集的问题。\n具体包括：', args_schema=<class 'textlong.tools.python_code.create_python_code_tool.<locals>.PythonCodeInput'>, func=<function create_python_code_tool.<locals>.python_code at 0x123522710>)]


In [14]:
from textlong.io import log

log(d.chat("今天广州天气如何啊"))

[TOOLS_CALL] {"index": 0, "id": "call_c8d2b05761b747e6afee86", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_c8d2b05761b747e6afee86", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天是晴天。 

广州今天的天气是晴朗的。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 53208238】


'广州今天的天气是晴朗的。'

In [15]:
log(d.chat("这是怎么得出来的？"))

我调用了get_current_weather这个工具函数来查询广州当前的天气情况，它返回的结果表明广州今天是晴天。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 73935956】


'我调用了get_current_weather这个工具函数来查询广州当前的天气情况，它返回的结果表明广州今天是晴天。'

In [15]:
log(d.chat("再看看上海"))

[TOOLS_CALL] {"index": 0, "id": "call_72521e92b59a461b897e73", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "上海\"}"}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_72521e92b59a461b897e73", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"上海\"}"}}}
上海今天是晴天。 

上海今天的天气也是晴朗的。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 22977051】


'上海今天的天气也是晴朗的。'

In [16]:
d.state

State(output=<None>, messages=<8>, data=<None>, knowledge=<>)

### 异步调用

In [4]:
from textlong.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.desk import Desk
from textlong.io import alog
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

messages = []
d = Desk(qwen, toolkits=[get_current_weather], tools=[convert_to_openai_tool(get_current_weather)])

In [6]:
await alog(d.achat("广州下雨不？"))

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_1bd5ab607b9c4f3aac2819", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_1bd5ab607b9c4f3aac2819", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天是晴天。 

广州今天是晴天，没有下雨。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 42846926】


'广州今天是晴天，没有下雨。'

## 写作提示语模板

In [17]:
from textlong.hub import load_chat_template, load_resource_chat_template

### 加载对话模板的资源

指定资源库中的模板ID即可加载模板。

In [18]:
load_chat_template("IDEA")

PromptTemplate(input_variables=['task'], template='你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n{{task}}\n', template_format='mustache')

In [19]:
load_chat_template("OUTLINE")

PromptTemplate(input_variables=['task'], template='你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 请按照需要输出一份写作提纲\n4. 提纲的构成应当由多个阔系额要求构成\n5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等\n5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹\n6. 你只能输出提纲，不要输出具体的扩写内容\n\n**你的任务是:**\n{{task}}\n\n\n**输出例子1**\n```\n# 第一章 标题1\n## 一、XXXX\n<OUTLINE>\n扩写摘要:\n对扩写内容做摘要总结\n\n扩写要求：\n- xxx\n- xxx\n</OUTLINE>\n\n## 二、XXXX\n<OUTLINE>\n</OUTLINE>\n\n## （更多扩写提纲）\n```\n\n', template_format='mustache')

### 查看构造出的提示语模板

In [20]:
print(load_chat_template("OUTLINE").format(task="请帮我写一首歌"))

你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出
3. 请按照需要输出一份写作提纲
4. 提纲的构成应当由多个阔系额要求构成
5. 扩写要求必须包含清晰描述，如预估字数、创意要点、创作思路、创作中涉及到的实体名称等
5. 每一个扩写要求由完整的 `<OUTLINE>`和`</OUTLINE>` 包裹
6. 你只能输出提纲，不要输出具体的扩写内容

**你的任务是:**
请帮我写一首歌


**输出例子1**
```
# 第一章 标题1
## 一、XXXX
<OUTLINE>
扩写摘要:
对扩写内容做摘要总结

扩写要求：
- xxx
- xxx
</OUTLINE>

## 二、XXXX
<OUTLINE>
</OUTLINE>

## （更多扩写提纲）
```




### 单轮写作

In [1]:
from textlong.llm import qwen, fake_llm
from textlong.desk import Desk
from textlong.io import log, alog

d = Desk(qwen)

In [2]:
log(d.write("帮我写一首关于兔子的四句儿歌", template="IDEA"))

小白兔白又白，  
耳朵长长跳得欢。  
爱吃萝卜和青菜，  
蹦蹦跳跳真可爱。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 75971425】


'小白兔白又白，  \n耳朵长长跳得欢。  \n爱吃萝卜和青菜，  \n蹦蹦跳跳真可爱。'

In [3]:
await alog(d.awrite("帮我写一首关于兔子的四句儿歌", template="IDEA"))

小白兔白又白，
耳朵长长跳得欢。
爱吃萝卜和青菜，
蹦蹦跳跳真可爱。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 36764839】


'小白兔白又白，\n耳朵长长跳得欢。\n爱吃萝卜和青菜，\n蹦蹦跳跳真可爱。'

In [23]:
log(d.chat("它喜欢吃什么？", k=1, llm=fake_llm))

[INFO] system: 你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出

**你的任务是:**
帮我写一首关于兔子的四句儿歌

[INFO] user: 请开始
[INFO] assistant: 小白兔白又白，  
耳朵长长跳得欢。  
爱吃萝卜和青菜，  
蹦蹦跳跳真可爱。
[INFO] user: 它喜欢吃什么？
这是一个模拟调用!

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 88267255】


'这是一个模拟调用!'

In [4]:
await alog(d.afrom_outline())

[INFO] 没有提纲可供扩写


''

In [25]:
d.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首关于兔子的四句儿歌\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '小白兔白又白，  \n耳朵长长跳得欢。  \n爱吃萝卜和青菜，  \n蹦蹦跳跳真可爱。'},
 {'role': 'user', 'content': '它喜欢吃什么？'},
 {'role': 'assistant', 'content': '这是一个模拟调用!'}]

### 单轮写作 + 回调工具

In [26]:
from textlong.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.desk import Desk
from textlong.io import log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天下大雨。 "

d = Desk(qwen, tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])
log(d.write({"task": "根据广州今天的天气写一首4句歌词"}, template="IDEA"))

[TOOLS_CALL] {"index": 0, "id": "call_1359345025244ee08210ab", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_1359345025244ee08210ab", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天下大雨。 

雨珠倾洒羊城间，  
湿漉街巷映云天。  
伞花朵开人潮中，  
共舞水珠谱新篇。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 55503433】


'雨珠倾洒羊城间，  \n湿漉街巷映云天。  \n伞花朵开人潮中，  \n共舞水珠谱新篇。'

In [27]:
d.output

'雨珠倾洒羊城间，  \n湿漉街巷映云天。  \n伞花朵开人潮中，  \n共舞水珠谱新篇。\n\n'

### 单轮写作 + 多轮对话

In [28]:
log(d.chat("改为8句吧"))

雨幕密织笼羊城，  
珠水潺潺和雨声。  
行人匆匆油纸伞，  
街头巷尾水墨情。  

乌云压顶天色暗，  
却映繁华灯火明。  
绿叶更翠吸甘霖，  
花儿含羞带雨睛。  

孩童笑语跳水坑，  
回忆童年那份真。  
雨后空气泛清新，  
广州雨天也醉人。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 35775428】


'雨幕密织笼羊城，  \n珠水潺潺和雨声。  \n行人匆匆油纸伞，  \n街头巷尾水墨情。  \n\n乌云压顶天色暗，  \n却映繁华灯火明。  \n绿叶更翠吸甘霖，  \n花儿含羞带雨睛。  \n\n孩童笑语跳水坑，  \n回忆童年那份真。  \n雨后空气泛清新，  \n广州雨天也醉人。'

In [29]:
d.messages

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n根据广州今天的天气写一首4句歌词\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'index': 0,
    'id': 'call_1359345025244ee08210ab',
    'type': 'function',
    'function': {'name': 'get_current_weather',
     'arguments': '{"location": "广州"}'}}]},
 {'role': 'tool', 'name': 'get_current_weather', 'content': '广州今天下大雨。 '},
 {'role': 'assistant',
  'content': '雨珠倾洒羊城间，  \n湿漉街巷映云天。  \n伞花朵开人潮中，  \n共舞水珠谱新篇。'},
 {'role': 'user', 'content': '改为8句吧'},
 {'role': 'assistant',
  'content': '雨幕密织笼羊城，  \n珠水潺潺和雨声。  \n行人匆匆油纸伞，  \n街头巷尾水墨情。  \n\n乌云压顶天色暗，  \n却映繁华灯火明。  \n绿叶更翠吸甘霖，  \n花儿含羞带雨睛。  \n\n孩童笑语跳水坑，  \n回忆童年那份真。  \n雨后空气泛清新，  \n广州雨天也醉人。'}]

In [30]:
from textlong.llm import fake_llm
log(d.chat("这是古典诗歌吗？", llm=qwen, k=1))

是的，这首诗采用了古典诗歌的风格和意象，每句四字或五字，描绘了广州雨天的景色与城市生活的温馨画面，融入了自然美与人文情感，体现了古典诗词的韵味。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 96455443】


'是的，这首诗采用了古典诗歌的风格和意象，每句四字或五字，描绘了广州雨天的景色与城市生活的温馨画面，融入了自然美与人文情感，体现了古典诗词的韵味。'

### 提纲 + 扩写

In [5]:
from textlong.llm import qwen, fake_llm
from textlong.desk import Desk
from textlong.io import log, alog

d = Desk(qwen)

In [6]:
await alog(d.awrite({
    "task": "帮我写一首关于兔子的儿歌，两段即可，第一段些兔子有哪些朋友，第二段些和朋友们一起玩（要求提到具体的朋友）"
}))

# 兔子的欢乐时光
## 一、兔子与它的朋友们
<OUTLINE>
扩写摘要:
介绍兔子在森林中的几位好朋友，强调友谊的多彩与温馨。

扩写要求：
- 预估字数：30字
- 创意要点：提到至少三种不同的动物作为兔子的朋友，每种动物简要描述其特征或习性，以增加趣味性和教育意义。
- 创作思路：通过欢快的节奏，展现一个和谐共处的动物世界，让孩子们感受到友情的美好。
- 实体名称：兔子、小鸟（爱唱歌）、小松鼠（爱收集坚果）、小青蛙（跳跳活泼）
</OUTLINE>

## 二、欢聚同乐日
<OUTLINE>
扩写摘要:
描绘兔子与朋友们一起玩耍的快乐场景，强调游戏的乐趣和团队合作。

扩写要求：
- 预估字数：30字
- 创意要点：描述一个具体的游戏或活动，如“躲猫猫”或“森林接力赛”，展示每个朋友如何利用自己的特长参与其中。
- 创作思路：使用富有节奏感的语言，营造出一种活泼、欢快的氛围，同时传达出团结协作的重要性。
- 实体名称：兔子（灵活领队）、小鸟（高空侦查）、小松鼠（快速攀爬）、小青蛙（池塘裁判）
</OUTLINE>

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 85962744】


'# 兔子的欢乐时光\n## 一、兔子与它的朋友们\n<OUTLINE>\n扩写摘要:\n介绍兔子在森林中的几位好朋友，强调友谊的多彩与温馨。\n\n扩写要求：\n- 预估字数：30字\n- 创意要点：提到至少三种不同的动物作为兔子的朋友，每种动物简要描述其特征或习性，以增加趣味性和教育意义。\n- 创作思路：通过欢快的节奏，展现一个和谐共处的动物世界，让孩子们感受到友情的美好。\n- 实体名称：兔子、小鸟（爱唱歌）、小松鼠（爱收集坚果）、小青蛙（跳跳活泼）\n</OUTLINE>\n\n## 二、欢聚同乐日\n<OUTLINE>\n扩写摘要:\n描绘兔子与朋友们一起玩耍的快乐场景，强调游戏的乐趣和团队合作。\n\n扩写要求：\n- 预估字数：30字\n- 创意要点：描述一个具体的游戏或活动，如“躲猫猫”或“森林接力赛”，展示每个朋友如何利用自己的特长参与其中。\n- 创作思路：使用富有节奏感的语言，营造出一种活泼、欢快的氛围，同时传达出团结协作的重要性。\n- 实体名称：兔子（灵活领队）、小鸟（高空侦查）、小松鼠（快速攀爬）、小青蛙（池塘裁判）\n</OUTLINE>'

In [6]:
for doc in d.state.markdown.documents:
    print(doc.metadata['id'])

2909-353-000
2909-055-001
2909-839-002
2909-809-003
2909-406-004
2909-887-005
2909-958-006


In [7]:
await alog(d.afrom_outline(llm=qwen))

[INFO] 执行扩写任务 <1898-638-002>：
扩写摘要:
介绍兔子在森林中的几位好朋友，强调友谊的多彩与温馨。

扩写要求：
- 预估字数：30字
- 创意要点：提到至少三种不同的动物作为兔子的朋友，每种动物简要描述其特征或习性，以增加趣味性和教育意义。
- 创作思路：通过欢快的节奏，展现一个和谐共处的动物世界，让孩子们感受到友情的美好。
- 实体名称：兔子、小鸟（爱唱歌）、小松鼠（爱收集坚果）、小青蛙（跳跳活泼）
## 一、兔子与它的朋友们

在葱郁的森林深处，兔子拥有一群快乐的伙伴。有爱在晨光中唱着悦耳歌曲的小鸟，它们的旋律总能让森林充满生机；小松鼠是位勤劳的坚果收藏家，蹦蹦跳跳间，为冬天储粮，它那毛茸茸的大尾巴，是森林中最温暖的风景；还有小青蛙，一蹦一跳活泼极了，它绿油油的身影在池塘边忽隐忽现，讲述着勇敢探险的故事。这三两知己，让兔子的每一天都充满了色彩与欢笑。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 40107261】
[INFO] 执行扩写任务 <1898-397-005>：
扩写摘要:
描绘兔子与朋友们一起玩耍的快乐场景，强调游戏的乐趣和团队合作。

扩写要求：
- 预估字数：30字
- 创意要点：描述一个具体的游戏或活动，如“躲猫猫”或“森林接力赛”，展示每个朋友如何利用自己的特长参与其中。
- 创作思路：使用富有节奏感的语言，营造出一种活泼、欢快的氛围，同时传达出团结协作的重要性。
- 实体名称：兔子（灵活领队）、小鸟（高空侦查）、小松鼠（快速攀爬）、小青蛙（池塘裁判）
## 二、欢聚同乐日

阳光洒满林间空地，兔子与朋友们开启了“森林接力赛”。兔子作为灵活领队，灵巧规划路线；小鸟高空侦查，指引方向；小松鼠在树木间快速攀爬，展示非凡技巧；小青蛙则在池塘边担任裁判，公正而欢腾。它们穿梭于自然之中，每一次接力都伴随着笑声与鼓励，共同绘制了一幅活力四射的友谊画卷。

[END] 【内容由幻蝶AI生成，其观点仅代表创作者个人立场，可登录 http://www.illufly.com 查验其校验码 73708393】


'## 一、兔子与它的朋友们\n\n在葱郁的森林深处，兔子拥有一群快乐的伙伴。有爱在晨光中唱着悦耳歌曲的小鸟，它们的旋律总能让森林充满生机；小松鼠是位勤劳的坚果收藏家，蹦蹦跳跳间，为冬天储粮，它那毛茸茸的大尾巴，是森林中最温暖的风景；还有小青蛙，一蹦一跳活泼极了，它绿油油的身影在池塘边忽隐忽现，讲述着勇敢探险的故事。这三两知己，让兔子的每一天都充满了色彩与欢笑。## 二、欢聚同乐日\n\n阳光洒满林间空地，兔子与朋友们开启了“森林接力赛”。兔子作为灵活领队，灵巧规划路线；小鸟高空侦查，指引方向；小松鼠在树木间快速攀爬，展示非凡技巧；小青蛙则在池塘边担任裁判，公正而欢腾。它们穿梭于自然之中，每一次接力都伴随着笑声与鼓励，共同绘制了一幅活力四射的友谊画卷。'

In [8]:
d.state.from_outline

{'1898-638-002': [{'role': 'system',
   'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 你必须根据已有提纲扩写，不要修改提纲中对扩写的要求和限定，不要额外发挥\n\n**已有文字草稿如下:**\n```markdown\n# 兔子的欢乐时光\n\n## 一、兔子与它的朋友们\n\n&lt;&lt;&lt;YOUR_TEXT&gt;&gt;&gt;\n\n\n## 二、欢聚同乐日\n\n...\n\n```\n\n**请你按照如下扩写任务要求生成一段文字，使其适合替换上面文字草稿中`<<<YOUR_TEXT>>>`所在位置:**\n```markdown\n扩写摘要:\n介绍兔子在森林中的几位好朋友，强调友谊的多彩与温馨。\n\n扩写要求：\n- 预估字数：30字\n- 创意要点：提到至少三种不同的动物作为兔子的朋友，每种动物简要描述其特征或习性，以增加趣味性和教育意义。\n- 创作思路：通过欢快的节奏，展现一个和谐共处的动物世界，让孩子们感受到友情的美好。\n- 实体名称：兔子、小鸟（爱唱歌）、小松鼠（爱收集坚果）、小青蛙（跳跳活泼）\n```\n'},
  {'role': 'user', 'content': '请开始'},
  {'role': 'assistant',
   'content': '## 一、兔子与它的朋友们\n\n在葱郁的森林深处，兔子拥有一群快乐的伙伴。有爱在晨光中唱着悦耳歌曲的小鸟，它们的旋律总能让森林充满生机；小松鼠是位勤劳的坚果收藏家，蹦蹦跳跳间，为冬天储粮，它那毛茸茸的大尾巴，是森林中最温暖的风景；还有小青蛙，一蹦一跳活泼极了，它绿油油的身影在池塘边忽隐忽现，讲述着勇敢探险的故事。这三两知己，让兔子的每一天都充满了色彩与欢笑。'}],
 '1898-397-005': [{'role': 'system',
   'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n3. 你必须根据已有提纲扩写，不要修改提纲中对扩写的要求和限定，不要额外发挥\n\n*

In [9]:
print(d.output)

# 兔子的欢乐时光

## 一、兔子与它的朋友们


在葱郁的森林深处，兔子拥有一群快乐的伙伴。有爱在晨光中唱着悦耳歌曲的小鸟，它们的旋律总能让森林充满生机；小松鼠是位勤劳的坚果收藏家，蹦蹦跳跳间，为冬天储粮，它那毛茸茸的大尾巴，是森林中最温暖的风景；还有小青蛙，一蹦一跳活泼极了，它绿油油的身影在池塘边忽隐忽现，讲述着勇敢探险的故事。这三两知己，让兔子的每一天都充满了色彩与欢笑。


## 二、欢聚同乐日


阳光洒满林间空地，兔子与朋友们开启了“森林接力赛”。兔子作为灵活领队，灵巧规划路线；小鸟高空侦查，指引方向；小松鼠在树木间快速攀爬，展示非凡技巧；小青蛙则在池塘边担任裁判，公正而欢腾。它们穿梭于自然之中，每一次接力都伴随着笑声与鼓励，共同绘制了一幅活力四射的友谊画卷。


